# 智能体的记忆

这个笔记本演示了如何在LangChain中为智能体(Agent)添加记忆功能，使其能够记住与用户之间的对话历史。

## 主要内容

本教程涵盖了以下内容：

1. **基本记忆实现**
   - 通过手动管理chat_history参数来实现记忆
   - 创建和维护对话历史数组

2. **高级记忆管理**
   - 使用RunnableWithMessageHistory自动跟踪对话历史
   - 配置会话ID以支持多用户场景

3. **实际应用演示**
   - 智能体能够记住用户的名字
   - 在后续对话中引用之前交流的信息

通过使用记忆功能，智能体可以提供更加连贯、个性化的用户体验，使对话更加自然流畅。这对于构建聊天机器人、虚拟助手和其他需要上下文感知的AI应用至关重要。

## 适用场景

- 多轮对话应用
- 个性化用户体验设计
- 需要记住用户偏好的智能助手
- 任何需要上下文连贯性的对话系统

通过本教程，您将学习如何让AI保持对话的上下文连贯性，并在长期交互中保持用户信息。

In [13]:
import os
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent,AgentExecutor
from langchain_core.messages import AIMessage,HumanMessage

# 初始化大模型
llm = ChatOpenAI(
    model = "qwen-turbo",
    api_key = os.getenv("API_KEY"),
    base_url = os.getenv("API_BASEURL")
)

prompt = hub.pull("hwchase17/openai-functions-agent")

# 创建智能体
agent = create_openai_functions_agent(llm,[],prompt)

# 创建智能体执行器
agent_executor = AgentExecutor(agent=agent, tools=[],verbose= True)

# 通过chat_history参数一起丢给大模型实现历史记忆
# 尝试对话
chat = agent_executor.invoke({"input": "你好,我是呱呱蛙？","chat_history":[]})

# print(chat)

# 创建历史记录数组
history = [] + [HumanMessage(content=chat['input']),AIMessage(content=chat['output'])]

# print(history)

chat2 = agent_executor.invoke({"input":"我是谁？","chat_history" : history})

# print(chat2)

# 使用RunnableWithMessageHistory实现自动跟踪

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_history = RunnableWithMessageHistory(
    runnable= agent_executor,
    get_session_history= lambda session_id : message_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

agent_with_history.invoke(
    {"input":"你好，我是呱呱蛙"},
    config={"configurable":{"session_id" : "s1"}}
)

agent_with_history.invoke(
    {"input":"我的名字是什么？"},
    config={"configurable":{"session_id" : "s1"}}
)


d:\Anaconda\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
你好，呱呱蛙！很高兴认识你。有什么我可以帮助你的？

> Finished chain.


> Entering new AgentExecutor chain...
哈哈，你就是呱呱蛙呀！听起来像是一个有趣的角色。你是从哪里冒出来的？有什么特别的故事想和我分享吗？ 😊

> Finished chain.


> Entering new AgentExecutor chain...
你好呱呱蛙！很高兴认识你。我是通义千问，一个大型语言模型。如果你有任何问题或需要帮助，尽管告诉我哦！

> Finished chain.


> Entering new AgentExecutor chain...
你的名字是“呱呱蛙”。有趣的名字！有什么我可以帮助你的吗？

> Finished chain.


{'input': '我的名字是什么？',
 'chat_history': [HumanMessage(content='你好，我是呱呱蛙', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好呱呱蛙！很高兴认识你。我是通义千问，一个大型语言模型。如果你有任何问题或需要帮助，尽管告诉我哦！', additional_kwargs={}, response_metadata={})],
 'output': '你的名字是“呱呱蛙”。有趣的名字！有什么我可以帮助你的吗？'}